<center><b><H1> KAGGLE COMPETITION GNR 638 </H1></b></center>
<center><b><H2> VINIT AWALE </H2></b></center>
<center><b><H2> 18D070067 </H2></b></center>
<center><b><H2> ELECTRICAL DUAL DEGREE </H2></b></center>

## Split the training folder to train and validation folders


In [4]:
import splitfolders

splitfolders.ratio('Dataset\\gnr6382021\\train', output='Dataset\\gnr6382021\\train', seed=1337, ratio=(.8, .2))  # 80% training, 20% validation

Copying files: 500 files [00:00, 584.89 files/s]


After this the folder structure is adjusted to have the following structure:


```
Dataset
├── train
│   ├──basketball_court
│   ├──bridge
│   ├──crosswalk   
│   ├──golf_course
│   ├──oil_well
│   ├──overpass
│   ├──railway
|   ├──runway
|   ├──swimming_pool
|   ├──tennis_court
|── val
│   ├──basketball_court
│   ├──bridge
│   ├──crosswalk
│   ├──golf_course
│   ├──oil_well
│   ├──overpass
│   ├──railway
|   ├──runway
|   ├──swimming_pool
|   ├──tennis_court
└── test


## Imports

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.utils.data.sampler import SubsetRandomSampler
